OpenCV MP4 I/O <h3>

In [7]:
import numpy as np
import datetime
import cv2 
import glob,os

def display_mp4(fullpath, starttime=0):
    cap = cv2.VideoCapture(fullpath)
    # get video info
    width  = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    fps       = cap.get(cv2.CAP_PROP_FPS)
    frame_seq = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    
    time_value_list = [int(i) for i in starttime.split(':')]
    starttime_sec = time_value_list[0]*3600 + time_value_list[1]*60 + time_value_list[2]
    frame_to_go = starttime_sec * int(fps)
    
    cap.set(1,frame_to_go)
    
    while(cap.isOpened()):
        ret, frame = cap.read()
        
        
        # change color
        """
        f_max_R, f_min_R = np.max(frame[:,:,0]), np.min(frame[:,:,0])
        f_max_G, f_min_G = np.max(frame[:,:,1]), np.min(frame[:,:,1])
        f_max_B, f_min_B = np.max(frame[:,:,2]), np.min(frame[:,:,2])
        #print (f_max, f_min)
        R_w, G_w, B_w = 1./3., 1./3., 1./3.
        c_ratio = f_max_R*R_w + f_max_G*G_w + f_max_B*B_w
        cv2.imshow('frame',frame/float(c_ratio))        
        """

        # change color space
        """
        #img_changed = cv2.cvtColor(frame, cv2.COLOR_BGR2XYZ)
        #img_changed = cv2.bilateralFilter(img_changed, d=5, sigmaColor=5, sigmaSpace=2)
        cv2.imshow('frame',img_changed)
        """

        # Laplacian edge detection
        """
        frame = cv2.GaussianBlur(frame, ksize=(3,3),sigmaX=1.5)
        frame = cv2.addWeighted(frame, 1.5, frame, 1.5, 3.)
        frame = cv2.Laplacian(frame,cv2.CV_32F)
        #frame =  cv2.convertScaleAbs(frame)
        cv2.imshow('frame',frame)
        """

        
        # Sobel edge detection
        #frame = cv2.addWeighted(frame, 1., frame, 1., 3.)
        frame = cv2.GaussianBlur(frame, ksize=(5,5),sigmaX=30)
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        #gray = cv2.addWeighted(gray,1.4 , gray, 1.4, 3.) 
        #
        gray_sobelx = cv2.Sobel(gray,cv2.CV_64F,1,0,ksize=5)
        gray_sobely = cv2.Sobel(gray,cv2.CV_64F,0,1,ksize=5)
        #change to unsigned 8 bit int
        gray_abs_sobelx = cv2.convertScaleAbs(gray_sobelx) 
        gray_abs_sobely = cv2.convertScaleAbs(gray_sobely)
        gray_sobel_edge = cv2.addWeighted(gray_abs_sobelx ,0.4 ,gray_abs_sobely ,0.4 ,0.) 
        cv2.imshow('gray_sobel_edge',gray_sobel_edge) 
        
        #cv2.waitKey(0)  # frame-by-frame waiting keystroke
        # waitKey val > 1000/25 (40), wait long, display slower than 25frame/sec
        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


#fullpath = "/Users/sclc/ds_videos/xinlizui.mp4"
#display_mp4(fullpath,"01:10:11")

In [8]:
#
"""
fullpath = "/Users/sclc/ds_videos/xinlizui.mp4"
cap = cv2.VideoCapture(fullpath)
ret, frame = cap.read()

print (cap.get(cv2.CAP_PROP_FPS))
print (cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS)
frame_num = cap.get(cv2.CAP_PROP_FRAME_COUNT)
len_video_sec = int (int(frame_num)/int(fps))
print ("video length "+str(datetime.timedelta(seconds=len_video_sec)))

#print(np.max(frame[:,:,0]), np.min(frame[:,:,0]))

#cv2.imshow('frame',frame/55.)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

"""

#
"""
length = cap.get(cv2.CAP_PROP_POS_FRAMES)
width  = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
fps    = cap.get(cv2.CAP_PROP_FPS)
print (length, width, height, fps)
"""

#
"""
# http://labs.eecs.tottori-u.ac.jp/sd/Member/oyamada/OpenCV/html/py_tutorials/py_imgproc/py_colorspaces/py_colorspaces.html
flags = [i for i in dir(cv2) if i.startswith('COLOR_')]
print ([i for i in flags if i.startswith("COLOR_BGR")])
"""



'\n# http://labs.eecs.tottori-u.ac.jp/sd/Member/oyamada/OpenCV/html/py_tutorials/py_imgproc/py_colorspaces/py_colorspaces.html\nflags = [i for i in dir(cv2) if i.startswith(\'COLOR_\')]\nprint ([i for i in flags if i.startswith("COLOR_BGR")])\n'

In [9]:


def get_im(path):

    #img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    img = cv2.imread(path)

    resized = img
    #resized = cv2.resize(img, (img_cols, img_rows))

    return resized

def read_train_data_fullname(filepath):

    
    files = glob.glob(filepath)
    # sorting
    filenames = sorted(files, key=lambda x: int(x.split("fig")[-1].split("_")[0]))
    
    return filenames

#filepath = "/Users/sclc/ds_images/figsTut_at52_20170823_a/*.png"
#filenames=read_train_data_fullname(filepath)
#for fl in filenames:
    #print(fl)
    #print(os.path.basename(fl))


In [10]:
def combine_to_video(filenames):
    
    frame = cv2.imread(filenames[0])
    width, height, channels = frame.shape
    # Define the codec and create VideoWriter object
    video_output = "./a.mp4"
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Be sure to use lower case
    out = cv2.VideoWriter(video_output, fourcc, 2.0, (width, height))
    
    for fl in filenames:
        frame = cv2.imread(fl)
        out.write(frame)
        
        cv2.imshow('video',frame)
        if (cv2.waitKey(1) & 0xFF) == ord('q'): # Hit `q` to exit
            break
    
    # Release everything if job is finished
    out.release()
    cv2.destroyAllWindows()
    
    # we need to know when the generation finished
    print("The output video is {}".format(video_output))

filepath = "/Users/sclc/ds_images/figsTut_at52_20170823_a/*.png"
filenames=read_train_data_fullname(filepath)
combine_to_video(filenames)

The output video is ./a.mp4
